# Fermi-Hubbard model (H=A+B+C)

The one-dimensional Fermi-Hubbard Hamiltonian

$$
H = - J \sum_{j = 1}^{L - 1} \sum_{\sigma \in \{ \uparrow, \downarrow \}} c_{j, \sigma}^\dagger c_{j + 1, \sigma} + \text{h.c.} + U \sum_{j} n_{j\uparrow} n_{j\downarrow}
$$

where $j = 1, ..., L$ denotes site/orbital and $\sigma \in \{ \uparrow, \downarrow \}$ denotes spin. 

Reference: [Trotter error with commutator scaling for the Fermi-Hubbard model](http://arxiv.org/abs/2306.10603)

In [12]:
import numpy as np
from scipy.linalg import expm
import openfermion as of

from trotter import *

fig_dir, data_dir = './figs', './data'

In [13]:
"""Define the Hubbard Hamiltonian by OpenFermion."""
class hubbard_openfermion:
    def __init__(self, nsites, U, J=-1.0, pbc=False, verbose=False):
        # Each site has two spins.
        self.n_qubits = 2 * nsites

        def fop_2_sparse(fops):
            return [of.get_sparse_operator(fop, n_qubits=self.n_qubits).todense() for fop in fops ]

        # One-body (hopping) terms.
        self.one_body_fops = [op + of.hermitian_conjugated(op) for op in (
                of.FermionOperator(((i, 1), (i + 2, 0)), coefficient=J) for i in range(self.n_qubits - 2))]
        self.one_body_L = len(self.one_body_fops)
        self.one_body_sparse = fop_2_sparse(self.one_body_fops)

        # Two-body (charge-charge) terms.
        self.two_body_fops = [
            of.FermionOperator(((i, 1), (i, 0), (i + 1, 1), (i + 1, 0)), coefficient=U)
            for i in range(0, self.n_qubits, 2)]
        self.two_body_sparse = fop_2_sparse(self.two_body_fops)

        self.h_fop = of.fermi_hubbard(1, nsites, tunneling=-J, coulomb=U, periodic=pbc)
        self.h_sparse = of.get_sparse_operator(self.h_fop)
        self.ground_energy, self.ground_state = of.get_ground_state(self.h_sparse)
        assert sum(self.one_body_fops) + sum(self.two_body_fops) == self.h_fop
        if verbose: 
            print('one_body_terms: \n', self.one_body_fops)
            print('one_body_L: ', self.one_body_L)
            # print('one_body[0]: \n', self.one_body_sparse[0])
            print('one_body[0]: \n', of.get_sparse_operator(self.one_body_fops[0]))
            # print('sparse two-body: \n', of.get_sparse_operator(sum(self.two_body_fops)))
            # print('sparse two-body[0]: \n', self.two_body_sparse[0])
            # print('ground energy: \n', self.ground_energy)
        # return ground_energy

        self.one_body_01 = [term for index, term in enumerate(self.one_body_fops) if index % 4 == 0 or index % 4 == 1]
        self.one_body_01_sparse = fop_2_sparse(self.one_body_01)
        # print(self.one_body_01)
        self.one_body_23 = [term for index, term in enumerate(self.one_body_fops) if index % 4 == 2 or index % 4 == 3]
        self.one_body_23_sparse = fop_2_sparse(self.one_body_23)
        # print(self.one_body_23)
        assert sum(self.one_body_01) + sum(self.one_body_23) == sum(self.one_body_fops)

        self.one_body_0 = [term for index, term in enumerate(self.one_body_fops) if index % 3 == 0]
        self.one_body_1 = [term for index, term in enumerate(self.one_body_fops) if index % 3 == 1]
        self.one_body_2 = [term for index, term in enumerate(self.one_body_fops) if index % 3 == 2]

        assert sum(self.one_body_0) + sum(self.one_body_1)  + sum(self.one_body_2) == sum(self.one_body_fops)

        self.one_body_0_sparse = [term for index, term in enumerate(self.one_body_sparse) if index % 3 == 0]
        self.one_body_1_sparse = [term for index, term in enumerate(self.one_body_sparse) if index % 3 == 1]
        self.one_body_2_sparse = [term for index, term in enumerate(self.one_body_sparse) if index % 3 == 2]


For this model, we directly use matrix

In [14]:
n = 4
# n = 5
r = 10000
epsilon = 1e-3
J, U = -1, 1

t_num = 100
t_list = np.logspace(-1, 3, num=t_num)
# t_list = np.logspace(1.5, 3, num=t_num)
hubbard = hubbard_openfermion(n, U, verbose=True)
h_group = [sum(hubbard.one_body_01_sparse), sum(hubbard.one_body_23_sparse), sum(hubbard.two_body_sparse)]
exact_U_list = [expm(-1j * t * sum(h_group)) for t in t_list]
# pf1_list = [op_error(matrix_power(unitary_matrix_product(h_group, t=t/r), r), exact_U_list[i]) for i, t in enumerate(t_list)]
data_r = {key:[] for key in ['r', 't', 'emp_pf1', 'emp_pf2', 'bnd_pf1', 'bnd_pf2', 'bnd_our']}
data_r['r'], data_r['t'] = r, t_list  

# print('Hamiltonian groups: ', h_group)

one_body_terms: 
 [-1.0 [0^ 2] +
-1.0 [2^ 0], -1.0 [1^ 3] +
-1.0 [3^ 1], -1.0 [2^ 4] +
-1.0 [4^ 2], -1.0 [3^ 5] +
-1.0 [5^ 3], -1.0 [4^ 6] +
-1.0 [6^ 4], -1.0 [5^ 7] +
-1.0 [7^ 5]]
one_body_L:  6
one_body[0]: 
   (4, 1)	(-1+0j)
  (6, 3)	(1+0j)
  (1, 4)	(-1+0j)
  (3, 6)	(1+0j)


In [11]:
## store data
# np.save(f"{data_dir}/Hubbard_Tri_n={n}_r={r}.npy", data_r)
# n = 4
# r = 10000
# data_r = np.load(f"{data_dir}/Hubbard_Tri_n={n}_r={r}.npy", allow_pickle=True).item()